<a href="https://colab.research.google.com/github/leopard8k/IRCC_Scraping/blob/master/IRCC_Basic_QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install html2text

In [5]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import tqdm

In [27]:
CND_SITE = 'https://www.canada.ca'
IRCC_SUFFIX='/en/immigration-refugees-citizenship/'
filter = re.compile('^' + IRCC_SUFFIX + '.*')
exclude_some = re.compile("#")
url = CND_SITE + IRCC_SUFFIX
scraped_uris={
              IRCC_SUFFIX: {
                  "url": CND_SITE + IRCC_SUFFIX,
               "broken": False,
               "visited": False,
               "children": {},
               "text":"",
               }
}

In [4]:
def get_hrefs(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  return set(sorted([a['href'] for a in soup.findAll('a', href=filter) if not exclude_some.search(a['href'])]))

In [28]:
def get_dict_hrefs(uri):
  for link in tqdm.tqdm(uri.copy()):
    if not uri[link].get("visited"):
      uri[link]["visited"] = True
      response = requests.get(CND_SITE+link)
      if response.status_code == 200:
        uri[link]["broken"] = False
        soup = BeautifulSoup(response.text, "html.parser")
        uri[link]["children"] = set(sorted([a['href'] for a in soup.findAll('a', href=filter) if not exclude_some.search(a['href'])]))
        ## uri[link]["text"] = soup
        uri.update({a:{} for a in uri[link]["children"] if a not in uri}) 
      else:
        uri[link]["broken"] = True      
  
  return uri

In [29]:
len(scraped_uris)

1

In [30]:
??scraped_uris

In [31]:
scraped_uris.update(get_dict_hrefs(scraped_uris))
# in my experience two iterations returns over 5000 links
#   one can experience with more if necessary
SCRAPE_DEPTH = 1
WEBCALLS_WITH_NO_BREAK = 30

for i in tqdm.trange(SCRAPE_DEPTH, desc="Depth:"):
  web_calls=0
  for suffix in tqdm.tqdm(scraped_uris.copy(), desc="URI: "):
    get_dict_hrefs(scraped_uris)
    # take a second between every few site calls to not flood the site
    web_calls += 1
    if web_calls >= WEBCALLS_WITH_NO_BREAK:
      time.sleep(0.2)
      web_calls = 0
len(scraped_uris)

Streaming output truncated to the last 5000 lines.

 71%|███████   | 4095/5784 [01:25<05:38,  4.99it/s]






 71%|███████   | 4096/5784 [01:25<05:39,  4.97it/s]






 71%|███████   | 4097/5784 [01:25<05:52,  4.78it/s]






 71%|███████   | 4098/5784 [01:25<05:44,  4.89it/s]






 71%|███████   | 4099/5784 [01:25<05:29,  5.11it/s]






 71%|███████   | 4100/5784 [01:26<04:42,  5.95it/s]






 71%|███████   | 4102/5784 [01:26<03:59,  7.02it/s]






 71%|███████   | 4104/5784 [01:26<03:35,  7.79it/s]






 71%|███████   | 4106/5784 [01:26<03:22,  8.27it/s]






 71%|███████   | 4108/5784 [01:26<03:07,  8.92it/s]






 71%|███████   | 4110/5784 [01:26<02:52,  9.71it/s]






 71%|███████   | 4112/5784 [01:27<02:44, 10.16it/s]






 71%|███████   | 4114/5784 [01:27<02:26, 11.39it/s]






 71%|███████   | 4116/5784 [01:27<02:42, 10.26it/s]






 71%|███████   | 4118/5784 [01:27<03:24,  8.13it/s]






 71%|███████   | 4120/5784 [01:28<03:07,  8.89it/s]






 71%|███████   | 412

KeyboardInterrupt: ignored

In [ ]:
scraped_uris = get_hrefs(url)
# in my experience two iterations returns over 5000 links
#   one can experience with more if necessary
SCRAPE_DEPTH = 3
WEBCALLS_WITH_NO_BREAK = 30

for i in tqdm.trange(SCRAPE_DEPTH, desc="Depth:"):
  new_uris = set()
  web_calls=0
  for suffix in tqdm.notebook.tqdm(scraped_uris, desc="URI: "):
    new_uris |= get_hrefs(CND_SITE+suffix)
    # take a second between every few site calls to not flood the site
    web_calls += 1
    if web_calls >= WEBCALLS_WITH_NO_BREAK:
      time.sleep(1)
      web_calls = 0
  scraped_uris |= new_uris

len(scraped_uris)

In [ ]:
count = 10000
with tqdm.notebook.tqdm(scraped_uris, desc="Downloading") as uris:
  for ahref in uris:
    uris.set_postfix({"URI":ahref}) 
    download_url = CND_SITE + ahref
    response = requests.get(download_url)
    if response.status_code == 200:
      count += 1
      urllib.request.urlretrieve(download_url,'./file-'+str(count)+'.html') 
      time.sleep(0.1)

In [ ]:
!mkdir dataIRCC
!for file in file*html;do html2text ${file} > dataIRCC/${file}.txt;done

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4


In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

## Document Store

Haystack finds answers to queries within the documents stored in a `DocumentStore`. The current implementations of `DocumentStore` include `ElasticsearchDocumentStore`, `FAISSDocumentStore`,  `SQLDocumentStore`, and `InMemoryDocumentStore`.

**Here:** We recommended Elasticsearch as it comes preloaded with features like [full-text queries](https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html), [BM25 retrieval](https://www.elastic.co/elasticon/conf/2016/sf/improved-text-scoring-with-bm25), and [vector storage for text embeddings](https://www.elastic.co/guide/en/elasticsearch/reference/7.6/dense-vector.html).

**Alternatives:** If you are unable to setup an Elasticsearch instance, then follow the [Tutorial 3](https://github.com/deepset-ai/haystack/blob/master/tutorials/Tutorial3_Basic_QA_Pipeline_without_Elasticsearch.ipynb) for using SQL/InMemory document stores.

**Hint**: This tutorial creates a new document store instance with Wikipedia articles on Game of Thrones. However, you can configure Haystack to work with your existing document stores.

### Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (eg., in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
# Recommended: Start Elasticsearch using Docker
#! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.9.2

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

## Preprocessing of documents

Haystack provides a customizable pipeline for:
 - converting files into texts
 - cleaning texts
 - splitting texts
 - writing them to a Document Store


In [ ]:

doc_dir = "dataIRCC"

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = convert_files_to_dicts(dir_path=doc_dir, split_paragraphs=False)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'text': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
#}
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Finder)

# Let's have a look at the first 3 entries:
print(dicts[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

## Initalize Retriever, Reader,  & Finder

### Retriever

Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use Elasticsearch's default BM25 algorithm

**Alternatives:**

- Customize the `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `DensePassageRetriever` to use different embedding models for passage and query (see Tutorial 6)

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
# Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes with SQLite document store.

# from haystack.retriever.sparse import TfidfRetriever
# retriever = TfidfRetriever(document_store=document_store)

### Reader

A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers.
With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

**Here:** a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

**Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)

**Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)

**Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"

#### FARMReader

In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

#### TransformersReader

In [ ]:
# Alternative:
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

### Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `ExtractiveQAPipeline` that combines a retriever and a reader to answer our questions.
You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelinesmd).

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

## Voilà! Ask a question!

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = pipe.run(query="can people with covid19 come to canada?", top_k_retriever=10, top_k_reader=5)
print_answers(prediction, details="minimal")